# PyKale Tutorial: PAH Diagnosis from Cardiac MRI (CMR) via a Multilinear PCA-based Pipeline
| [Open in Colab](https://colab.research.google.com/github/pykale/pykale/blob/main/examples/cmri_mpca/tutorial.ipynb) (click `Runtime` → `Run all (Ctrl+F9)` |  [Launch Binder](https://mybinder.org/v2/gh/pykale/pykale/HEAD?filepath=examples%2Fcmri_mpca%2Ftutorial.ipynb) (click `Run` → `Run All Cells`) |

## Overview

- Pre-processing:
    - Registration
    - Masking
    - Rescaling
    - Normalization
- Machine learning pipeline:
    - Multilinear principal component analysis
    - Discriminative feature selection
    - Linear classification model training    

[See the full pipeline illustration for more details.](https://oup.silverchair-cdn.com/oup/backfile/Content_public/Journal/ehjcimaging/22/2/10.1093_ehjci_jeaa001/2/m_jeaa001f2.jpeg?Expires=1631272906&Signature=PKl6KLDSoNyiTy~GNtXayJCucGKhweXvGz~svHY~ThjqjbokMVCnyJMMjoGQ4C81HdUcdnJA-rcNaqmDjspUs5eAX7avG~ckkIGXqGbPWrfnaMfwywWG3EXvvH0tw9ZcFeelnWgF4lkT0RFDVgzvzhHBvefNgS0ZGwLqiGJduANJFmWIXvYgNiU6M6kRbdpOJZBltknUO~Jv43-ghqmEX7dTfOKAx6~14quDC5cgzVFfwNFRKSn0P5JZRf~wIhQ6GQ4tprl7eXuzuRHcKnFbM~UkZOtcQvVhJofCCgSDnExyS6bns9Dop39OlfQHUdY4cwn1WaSnMKEqAqQaKZ715w__&Key-Pair-Id=APKAIE5G5CRDK6RD3PGA)

**Reference:**

Swift, A. J., Lu, H., Uthoff, J., Garg, P., Cogliano, M., Taylor, J., ... & Kiely, D. G. (2021). [A machine learning cardiac magnetic resonance approach to extract disease features and automate pulmonary arterial hypertension diagnosis](https://academic.oup.com/ehjcimaging/article/22/2/236/5717931). European Heart Journal-Cardiovascular Imaging.

## Setup

In [ ]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    !pip uninstall --yes imgaug && pip uninstall --yes albumentations && pip install git+https://github.com/aleju/imgaug.git
    !git clone https://github.com/pykale/pykale.git
    %cd pykale
    !pip install .[image,example] 
    %cd examples/cmri_mpca
else:
    print('Not running on CoLab')

This imports required modules.

In [ ]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from config import get_cfg_defaults

from kale.utils.download import download_file_by_url
from kale.loaddata.image_access import dicom2arraylist, read_dicom_dir
from kale.interpret import visualize

## Get CMR Images, Landmark Locations, and Labels

In [ ]:
cfg_path = "configs/tutorial_svc.yaml" # Path to `.yaml` config file

cfg = get_cfg_defaults()
cfg.merge_from_file(cfg_path)
cfg.freeze()
print(cfg)

### Download Data

In [ ]:
base_dir = cfg.DATASET.BASE_DIR
file_format = cfg.DATASET.FILE_FORAMT
download_file_by_url(cfg.DATASET.SOURCE, cfg.DATASET.ROOT, "%s.%s" % (base_dir, file_format), file_format)

### Read DICOM Images

In [ ]:
img_path = os.path.join(cfg.DATASET.ROOT, base_dir, cfg.DATASET.IMG_DIR)
patient_dcm_list = read_dicom_dir(img_path, sort_instance=True, sort_patient=True)
images, patient_ids = dicom2arraylist(patient_dcm_list, return_patient_id=True)
patient_ids = np.array(patient_ids, dtype=int)
n_samples = len(images)

mask_path = os.path.join(cfg.DATASET.ROOT, base_dir, cfg.DATASET.MASK_DIR)
mask_dcm = read_dicom_dir(mask_path, sort_instance=True)
mask = dicom2arraylist(mask_dcm, return_patient_id=False)[0][0, ...]

### Read Landmarks and Get Labels

In [ ]:
landmark_path = os.path.join(cfg.DATASET.ROOT, base_dir, cfg.DATASET.LANDMARK_FILE)
landmark_df = pd.read_csv(landmark_path, index_col="Subject").loc[patient_ids]  # read .csv file as dataframe
landmarks = landmark_df.iloc[:, :-1].values
y = landmark_df["Group"].values
y[np.where(y != 0)] = 1  # convert to binary classification problem, i.e. no PH vs PAH

### Visualizing Data and Landmarks

In [ ]:
# Get landmark names from column names
marker_names = list(landmark_df.columns[1::2])
markers = []
for marker in marker_names:
    marker_name = marker.split(" ")
    marker_name.pop(-1)
    marker_name = " ".join(marker_name)
    markers.append(marker_name)

In [ ]:
# plot the first phase of images with landmarks

n_img_per_fig = 35
n_figures = int(n_samples / n_img_per_fig) + 1
for k in range(n_figures):
    visualize.plot_multi_images(
        [images[i][0, ...] for i in range(k * n_img_per_fig, min((k + 1) * n_img_per_fig, n_samples))],
        marker_locs=landmarks[k * n_img_per_fig: min((k + 1) * n_img_per_fig, n_samples), :],
        im_kwargs=dict(cfg.PLT_KWS.IM),
        marker_cmap="Set1",
        marker_kwargs=dict(cfg.PLT_KWS.MARKER),
        marker_titles=markers,
        image_titles=list(patient_ids[k * n_img_per_fig: min((k + 1) * n_img_per_fig, n_samples)]),
        n_cols=5,
    ).show()

## CMR Pre-processing

In [ ]:
from kale.prepdata.image_transform import mask_img_stack, normalize_img_stack, reg_img_stack, rescale_img_stack

### Image Registration

In [ ]:
img_reg, max_dist = reg_img_stack(images.copy(), landmarks, landmarks[0])

In [ ]:
plt_kawargs = {**{"im_kwargs": dict(cfg.PLT_KWS.IM), "image_titles": list(patient_ids)}, **dict(cfg.PLT_KWS.PLT)}
visualize.plot_multi_images([img_reg[i][0, ...] for i in range(n_samples)], **plt_kawargs).show()

### Masking

In [ ]:
img_masked = mask_img_stack(img_reg.copy(), mask)

In [ ]:
visualize.plot_multi_images([img_masked[i][0, ...] for i in range(n_samples)], **plt_kawargs).show()

### Rescaling

In [ ]:
img_rescaled = rescale_img_stack(img_masked.copy(), scale=1 / cfg.PROC.SCALE)

In [ ]:
visualize.plot_multi_images([img_rescaled[i][0, ...] for i in range(n_samples)], **plt_kawargs).show()

### Normalization

In [ ]:
img_norm = normalize_img_stack(img_rescaled.copy())

In [ ]:
visualize.plot_multi_images([img_norm[i][0, ...] for i in range(n_samples)], **plt_kawargs).show()

## PAH Classification

In [ ]:
from sklearn.model_selection import cross_validate
from kale.pipeline.mpca_trainer import MPCATrainer

x = np.concatenate([img_norm[i].reshape((1,) + img_norm[i].shape) for i in range(n_samples)], axis=0)
trainer = MPCATrainer(classifier=cfg.PIPELINE.CLASSIFIER, n_features=200)
cv_results = cross_validate(trainer, x, y, cv=10, scoring=["accuracy", "roc_auc"], n_jobs=1)

In [ ]:
cv_results

In [ ]:
print("Averaged training time: {:.4f} seconds" .format(np.mean(cv_results['fit_time'])))
print("Averaged testing time: {:.4f} seconds" .format(np.mean(cv_results['score_time'])))
print("Averaged Accuracy: {:.4f}" .format(np.mean(cv_results["test_accuracy"])))
print("Averaged AUC: {:.4f}" .format(np.mean(cv_results["test_roc_auc"])))

## Model Interpretation

In [ ]:
from kale.interpret import model_weights

trainer.fit(x, y)

weights = trainer.mpca.inverse_transform(trainer.clf.coef_) - trainer.mpca.mean_
weights = rescale_img_stack(weights, cfg.PROC.SCALE)  # rescale weights to original shape
weights = mask_img_stack(weights, mask)  # masking weights
top_weights = model_weights.select_top_weight(weights, select_ratio=0.02)  # select top 2% weights
visualize.plot_weights(
    top_weights[0][0],
    background_img=images[0][0],
    im_kwargs=dict(cfg.PLT_KWS.IM),
    marker_kwargs=dict(cfg.PLT_KWS.WEIGHT),
).show()